In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('covid_toy.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [3]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),
                                                df['has_covid'],
                                                test_size=0.2
                                                )
x_train.shape

(80, 5)

### 1. Aam Zindagi (without using column transformer)

In [5]:
# adding simple imputer to fever col
from sklearn.impute import SimpleImputer

si = SimpleImputer()

x_train_fever = si.fit_transform(x_train[['fever']])
x_test_fever = si.transform(x_test[['fever']])
x_train_fever.shape

(80, 1)

In [6]:
# Ordinalencoding -> cough
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder(categories=[['Mild','Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])
x_test_cough = oe.transform(x_test[['cough']])
x_train_cough.shape

(80, 1)

In [7]:
# OneHotEncoding -> gender,city
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False,drop='first')

x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city = ohe.transform(x_test[['gender','city']])
x_train_gender_city.shape

# 4 columns are showing becoz of drop first 

(80, 4)

In [8]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [9]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)

x_train_transformed.shape

(80, 7)


### Mentos Zindagi - column transformers


In [10]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[
    ('tf1',SimpleImputer(),['fever']),
    ('tf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [11]:
transformer.fit_transform(x_train).shape

(80, 7)